# Multiple linear regressions

In the previous subject, we learned about simple linear regressions which use the equation of a line to model the relationship between a single feature and the target variable. We will now see how to generalize this idea to multiples features.

At the end of this unit, **you should be able to fit linear regressions to data sets with multiple features using the lstsq() function from the Scipy library.**

In [1]:
import pandas as pd

# Load data
data_df = pd.read_csv('marketing-campaign.csv')
print('data_df shape:', data_df.shape) # (50, 4)
data_df.head()

data_df shape: (50, 4)


,tv,web,radio,sales
0,0.916,1.689,0.208,1.204
1,9.359,1.706,1.071,4.800
2,5.261,2.538,2.438,3.970
3,8.682,2.092,1.283,5.212
4,11.736,1.660,1.800,5.993


### Implementation with Scipy

In [2]:
# Extract input matrix X
X = data_df.drop('sales', axis=1).values
print('X:', X.shape) # (50, 3)

# Extract target vector y
y = data_df.sales.values
print('y:', y.shape) # (50,)

X: (50, 3)
y: (50,)


We now want to **find the vector of coefficients** 
⃗
w
 that minimizes an objective function. In this unit, we will use the lstsq() function from Scipy which computes the least squares solution of the equation 
A
x
=
b
. In our case, 
A
, 
b
 and 
x
 correspond respectively to 
X
, 
y
 and 
⃗
w
, and the least squares solution is the vector 
⃗
w
 that minimizes the squared distances between the two sides of the equation

X
⃗
w
=
y
**In other words, the lstsq() function will return the parameter values that minimize the squares of the difference between the predictions of our linear regression model (without the intercept term)** 
⃗
y
p
r
e
d
=
X
⃗
w
 and the target values 
y
.

Let's try it with our X and y arrays.

In [3]:
from scipy.linalg import lstsq

# Fit a multiple linear regression
w, rss, _, _ = lstsq(X, y)
print('w:', w) # [ 0.3958359   0.47521518  0.31040001]
print('RSS:', rss) # 1.688

w: [0.3958359  0.47521518 0.31040001]
RSS: 1.6884039033000033


The function returns four values. As for now, we will only look at the first two. The first one w is the vector of coefficients (one for each feature) and the second one rss is the residual sum of squares.

For reference, **the RSS score of our simple linear regression model was around 15.7. Hence, we gained a lot in accuracy by bringing the two other marketing budgets in the equation.**

### Adding the intercept term
The code from above computes the optimal solution without the intercept term 
w
0
. However, it's possible to make the lstsq() function compute it using a little trick. The idea is to add a column of ones in the matrix 
X
. This column corresponds to the 
w
0
 element in 
⃗
w
.

The ones multiply the intercept term 
w
0
, and we get the equation of linear regressions from above. To generate this additional column, we can use the ones() function from Numpy. Then, we concatenate it to the X matrix with the Numpy c_ object.




In [4]:
import numpy as np

# Add a column of ones
X1 = np.c_[
    np.ones(X.shape[0]), # Vector of ones of shape (n,)
    X # X matrix of shape (n,p)
]

X1[:5, :]

array([[ 1.   ,  0.916,  1.689,  0.208],
       [ 1.   ,  9.359,  1.706,  1.071],
       [ 1.   ,  5.261,  2.538,  2.438],
       [ 1.   ,  8.682,  2.092,  1.283],
       [ 1.   , 11.736,  1.66 ,  1.8  ]])

In [5]:
w, rss, _, _ = lstsq(X1, y)

In [6]:
print('w:', w)
print('RSS:', rss)

w: [0.02487092 0.39465146 0.47037002 0.30669954]
RSS: 1.6854508680824727


The array w has now four elements. The first one w[0] corresponds to the intercept term and the other three w[1:] to the coefficients. Note that the intercept is close to zero. Hence the RSS score didn't change significantly.

In [7]:
# Compute predictions
y_pred = np.matmul(X1, w)
print('y_pred:', y_pred.shape)

y_pred: (50,)


In [8]:
# Verify RSS score
def RSS(y, y_pred):
    return np.sum(np.square(np.subtract(y, y_pred)))

rss = RSS(y, y_pred)
print('RSS:', rss) 

RSS: 1.685450868082472


### Summary
In this unit, we saw how to generalize linear regressions to multiple features, and how to implement them using the lstsq() function from Scipy. In practice, we usually work with higher level tools such as the LinearRegression object from Scikit-learn. However, it's good to know how they work internally.

In the next unit, we will learn about the coefficient of determination 
R
2
 which is a common evaluation metric for linear regressions.

# R^2 coefficient
In this unit, we will learn about the coefficient of determination 
R
2
 which is a common evaluation metric for linear regressions. It's also the default metric returned by many Scikit-learn objects such as SGDRegressor or HuberRegressor.

R
2
 coefficient
Unlike cost functions, the coefficient 
R
2
 **doesn't directly express the error made by our model. In fact, we can interpret it as a comparison between our model and the constant mean baseline.**

In [9]:
import pandas as pd

# Load data
data_df = pd.read_csv('marketing-campaign.csv')
X = data_df.drop('sales', axis=1).values
y = data_df.sales.values


In [10]:
import numpy as np

# Define RSS measure
def RSS(y, y_pred):
    return np.sum(np.square(np.subtract(y, y_pred)))

# RSS of the baseline
rss_baseline = RSS(y, y.mean())
print('RSS baseline:', rss_baseline)

RSS baseline: 100.86060792000002


In [11]:
from scipy.linalg import lstsq

# Fit a multiple linear regression
X1 = np.c_[np.ones(X.shape[0]), X]
w, model_rss, _, _ = lstsq(X1, y)
print('RSS:', model_rss)

RSS: 1.6854508680824727


As we can see, there is a significant difference in the RSS scores between our baseline and the linear regression. To quantify this difference, we can compute the 
R
2
 coefficient which is one minus the ratio between the two scores.
 
- If R2= 1, then the ratio is zero which means that the RSS of the model is zero and that it makes no error
- If 
R
2
 is close to 
1
, then the model performs way better than the baseline
- If 
R
2
 is close to 
0
, then the model and the baseline have a similar performance
- If 
R
2
<
0
, then it performs worse than the baseline

In [12]:
# R^2 coefficient
R2 = 1 - (model_rss / rss_baseline)
print('R^2 coefficient:', R2)

R^2 coefficient: 0.9832893048848236


In [13]:
# R^2 of simple linear regression model
R2 = 1 - (15.74 / rss_baseline)
print('R^2 coefficient:', R2) 

R^2 coefficient: 0.8439430385697798


### Proportion of variance explained

*The coefficient of determination 
R
2
 is the proportion of the variance in the target variable 
y
 that is predictable from the input variable(s) 
x*

In other words, it's a measure of how well we can explain, with our model, how the target variable 
y
 varies using the input variable(s) 
x
. The idea is that the target values vary around their mean value and we can quantify this variation by summing the squares of the differences between each target value 
y
i
 and the mean target value 
¯
y
.

The ratio between these two values correspond to the **proportion of variance left in the data or not explained by the model**. If we subtract this ratio to one, we obtain the proportion of variance explained by our model which is the 
R
2
 coefficient.
 
 
### Summary
Here are a few takeaways about the 
R
2
 coefficient.

- Intuitively, it measures how well a model performs compared to the constant mean baseline
- It is always smaller than or equal to one. Ideally, it should be close to one
- The coefficient is defined as the proportion of variance explained by the model

So far, we focused on how the linear regression model works, but not on the methods that find the set of optimal parameters 
⃗
w
. **In the next unit, we will learn about the ordinary least squares (OLS) method which is the one used by the polyfit() and the lstsq() functions.**

# Ordinary least squares

The goal of this unit is to get familiar with the topic of cost function optimization. We will start with a review of derivatives and gradients, and see how we can use them to optimize cost functions.

In the second part of this unit, we will learn about the ordinary least squares (OLS) method which is the one used by the lstsq() function to compute the least squares solution. We will implement this method using basic Numpy code and get the set of optimal parameters of a linear regression model for the marketing campaign data set.

### Cost functions and derivatives

Gradient of the cost function
Ordinary least squares

Solving the equation on the gradient 
∇
f
=
0
 only works for a specific class of functions 
f
 that are "flat" only at their optimal value. The good news is that the RSS cost function of a linear regression model is one of them and we can find the optimal parameters analytically by solving the equation. You can take a look at this post if you want to learn more about the proof.

This analytical solution is called the ordinary least squares (OLS) solution. Here is the formula that we get when solving the equation on the gradient.

### OLS on the marketing campaign data set
We will now implement the OLS method using the formula from above. Our goal is to find the set of optimal parameters of a linear regression model for the marketing campaign data set.

Let's start by loading the data set into an X and a y Numpy array.

In [14]:
import pandas as pd

# Load data
data_df = pd.read_csv('marketing-campaign.csv')
X = data_df.drop('sales', axis=1).values
y = data_df.sales.values

We can then use the matmul() function from Numpy to compute matrix multiplications and get the transpose 
X
⊺
 with the T attribute of Numpy arrays. For the matrix inversion 
(
X
⊺
X
)
−
1
, we can use the Numpy inv() function from its linalg module.

In [15]:
# Compute OLS solution
XX = np.matmul(X1.T, X1)
Xy = np.matmul(X1.T, y)
w = np.matmul(np.linalg.inv(XX), Xy)

print('w:', w)

w: [0.02487092 0.39465146 0.47037002 0.30669954]


In [16]:
# Let's do a quick verification using the lstsq() function from Numpy.
from scipy.linalg import lstsq

# Verify with Scipy lstsq
w, _, _, _ = lstsq(X1, y)

print('w:', w)
# Prints: [ 0.02487092  0.39465146  0.47037002  0.30669954]

w: [0.02487092 0.39465146 0.47037002 0.30669954]


### Summary
In this unit, we learned about the ordinary least squares (OLS) solution which is the standard way to find the set of parameters for linear regressions. In particular, we saw that it's an analytical solution that minimizes the RSS measure by solving an equation on its gradient.

# Scikit-learn

In [17]:
import pandas as pd

# Load data
data_df = pd.read_csv('marketing-campaign.csv')
X = data_df.drop('sales', axis=1).values
y = data_df['sales'].values

The Scikit-learn library provides a **LinearRegression object** that uses the OLS method to fit a linear regression model. Just like the HuberRegressor and the SGDRegressor ones, this object **implements the fit() function.** In the Scikit-learn jargon, these objects are estimators and the function is part of the estimator API.

In [18]:
from sklearn.linear_model import LinearRegression

# Create a linear regression object
lr = LinearRegression()

# Fit the model
lr.fit(X, y)

# Print coefficients
print('Coefficients:', lr.coef_)
# Prints: [ 0.39465146  0.47037002  0.30669954]

print('Intercept:', lr.intercept_)
# Prints: 0.02487091788819562

Coefficients: [0.39465146 0.47037002 0.30669954]
Intercept: 0.024870917888195176


In this code, we start by creating a LinearRegression() object. Then, we call fit() with the X, y variables. This function also relies on the lstsq() function from Scipy to compute the set of optimal parameters. However, note that we don't have to add the additional column of ones to the input matrix X. The LinearRegression() object will automatically compute the intercept term 
w
0
. We can access the parameters with the coef_ and the intercept_ attributes.

For reference, here are the optimal coefficients that we computed in the last unit with the OLS formula.

For reference: scipy lstsq returns
w: [ 0.02487092  0.39465146  0.47037002  0.30669954]

### Predict and score functions

In [19]:
# Compute predictions
y_pred = lr.predict(X)
y_pred[:3]


array([1.24462012, 4.84934038, 4.04266482])

In [20]:
# We can also manually compute these predictions with the matmul() function from Numpy.
import numpy as np

y_pred = np.matmul(X, lr.coef_) + lr.intercept_
y_pred[:3]


array([1.24462012, 4.84934038, 4.04266482])

In [21]:
# Compute the R2 cofficient
R2 = lr.score(X, y)
print('R2:', R2)

R2: 0.9832893048848236


### SGDRegressor
The OLS method is the most common way to find the parameters of a linear regression model that minimize the squares of the residuals. However, in the unit about Huber loss, **we saw the SGDRegressor object which implements a variant of the gradient descent (GD) algorithm.**

It's important to understand that **this algorithm doesn't compute an analytical solution.** It's an iterative algorithm that tries to get closer to the optimal solution after each iteration. However, unlike the OLS method, gradient descent is very generic and can optimize many different cost functions, e.g., Huber loss.

To minimize the squares of the residuals, we can set its loss parameter to squared_loss. By default, it adds a penalization term to the cost function. We will learn more about it later in this course. As for now, we can set it 'none'.

In [22]:
from sklearn.linear_model import SGDRegressor

# Create the SGDRegressor object
lr_sgd = SGDRegressor(
    loss='squared_loss', # Cost function
    penalty='none', # Add a penalty term?
    max_iter=1000, # Number of iterations
    random_state=0, # The implementation shuffles the data
    tol=1e-3 # Tolerance for improvement (stop SGD once loss is below)
)

# Fit the linear regression model
lr_sgd.fit(X, y)

# Print coefficients
print('Coefficients:', lr_sgd.coef_)

print('Intercept:', lr_sgd.intercept_)

Coefficients: [0.39968853 0.44409771 0.25894341]
Intercept: [0.12807209]


The implementation of the SGDRegressor object shuffles the data before running the optimization algorithm. To get the results from above, you should set its random_state parameter to zero.

In [23]:
# Compute R2 coefficient
R2_sgd = lr_sgd.score(X, y)
print('R2_sgd:', R2_sgd)

R2_sgd: 0.9821546772612869


### Huber loss
We usually fit linear regressions using the least squares approach, i.e., minimizing the squares of the residuals. However, it's also possible to use other objective functions such as Huber loss.

To achieve this, we can create an HuberRegressor object which also implements the estimator interface.

In [24]:
from sklearn.linear_model import HuberRegressor

# Create the estimator
huber = HuberRegressor(epsilon=1.35)

# Fit it to X,y
huber.fit(X, y)

print('Coefficients:', huber.coef_)


print('Intercept:', huber.intercept_)


print('R^2 coefficient:', huber.score(X, y))


Coefficients: [0.39172544 0.4788203  0.29315421]
Intercept: 0.04586298819194167
R^2 coefficient: 0.983070157114285


 this code, we create a HuberRegressor object with a threshold of 1.35 and fit it to the X, y data. We can then access the parameters with the coef_ and the intercept_ attributes. They are a bit different than the ones from above because they optimize another cost function. Finally, we compute the 
R
2
 coefficient with the score() function.
 
### Summary
Let's summarize what we've learned in this unit. Here are a few takeaways.

- The Scikit-learn library implements several estimators to fit linear regressions, e.g., the LinearRegression, SGDRegressor and HuberRegressor objects
- They implement the estimator API, e.g., the fit(), predict(), score() functions.
- Scikit-learn estimators work with 2-dimensional arrays of features X, i.e., unlike the polyfit() function which expects a 1-dimensional input vector x.

In the next unit, we will do a summary of the different functions and objects to fit linear regressions, and we will see when to use each method.

# How to choose the appropriate method?

We've seen many different functions and objects to fit linear regressions. Let's do a summary and see when to use each method.

### Numpy polyfit
The polyfit(x, y, deg) function fits a polynomial of degree deg to a set of x/y points. The function computes the least squares solution which corresponds to a simple linear regression model when deg=1. Later in this course, we will see that this function is a special case of linear regressions with a polynomial basis, and we will see how to implement it with the LinearRegression and the PolynomialFeatures objects from Scikit-learn.

**Use the polyfit() function when fitting a polynomial between a single input variable x and an output variable y.**

### OLS: Numpy implementation
The ordinary least squares (OLS) formula is an analytical solution of linear regressions with the RSS cost function. It's possible to implement the formula with the Numpy matmul() and inv() functions.

⃗
w
=
(
X
⊺
X
)
−
1
X
⊺
⃗
y

**However, in practice, we don't implement this formula by ourselves but use existing optimized implementations.**

### OLS: Scipy lstsq
There are many ways to implement the OLS solution. You can take a look at this post to learn more about them. The lstsq() function from Scipy uses optimized algorithms to compute it.

### Scikit-learn LinearRegression
Scikit-learn provides a LinearRegression estimator to compute the OLS solution which uses the lstsq() function internally. It's very convenient to work with this object since it implements the estimator API. For this reason, **we recommend you to use this object to fit linear regressions.**

### Scikit-learn HuberRegressor
Scikit-learn also provides a HuberRegressor estimator for linear regressions with Huber loss. The object optimizes this cost function using a iterative method called BFGS.

**It's a good idea to try fitting a linear regression with Huber loss if you believe that there are outliers in the data.**

### Scikit-learn SGDRegressor
It's also possible to fit linear regressions with the SGDRegressor estimator from Scikit-learn which implements the **stochastic gradient descent (SGD) algorithm.** In short, it's a variant of the gradient descent (GD) algorithm that **can optimize many different functions, e.g., Huber loss, squared loss.** We will learn more about this algorithm in the next subject.

One of its main advantages is its complexity - **it is faster for very large data sets (e.g., millions of data points or tens of thousands of features) and requires less memory.** You can take a look at this tutorial from Scikit-learn if you want to learn more about it.

**Use it for linear regressions when it's not possible to use the LinearRegression or the HuberRegressor objects.**

# Ill-conditioning

In this unit, we will learn about **ill-conditioning** which can cause numerical issues when computing the ordinary least squares solution (OLS). We will start by describing **collinearity** which is a phenomenon closely related to ill-conditioning. Then, we will see how nearly collinear features can cause numerical issues. Finally, we will experiment with regularization which is a way to handle ill-conditioning.

### Collinearity

Collinearity (and multicollinearity) happens when there is an **exact linear relationship between one or more features in the input matrix 
X**
. This means that one feature is a combination of the others plus some constant. This may sound abstract, so let's take an example.

In [2]:
import pandas as pd

# Load data
data_df = pd.read_csv('bike-sharing.csv')

# Create Numpy arrays
temp = data_df.temp.values
users = data_df.users.values

# First five rows
data_df.head()

,temp,users
0,0.1964,120
1,0.2000,108
2,0.2270,82
3,0.2043,88
4,0.1508,41


As we can see, temperatures are not in degrees Celcius or Fahrenheit. In fact, the data set web page states that the original temperatures in degrees Celcius temp_C were rescaled between zero and one using the formula temp=(temp_C+8)/47.

In [3]:
# Create collinear feature
temp_C = 47*temp - 8

By construction, **the temp and the temp_C variables are collinear**. The issue with collinear features is that they make the 
X
 matrix with the additional column of ones, and its moment matrix 
X
⊺
X
, rank deficient which means that not all columns are linearly independent. A matrix that is rank-deficient is not invertible. **Hence, the OLS solution does not exist.**

In [4]:
import numpy as np

# Create input matrix X
X = np.c_[temp, temp_C]

# Add a column of ones
X1 = np.c_[np.ones(X.shape[0]), X]

# Compute rank
rank = np.linalg.matrix_rank(X1)
print('Rank', rank) # Returns: 2

Rank 2


In this code, we create the input matrix X with the collinear variables temp and temp_C. We then add a column of ones and end up with an input matrix X1 with three columns. **This matrix is rank-deficient because its number of independent features is 2. This number is called the rank of the matrix, and we say that X1 is rank-deficient because its rank is not maximal, i.e., it's not 3.**

### Collinearity in practice

**In theory, the OLS solution doesn't exist when 
X
 contains collinear features. However, most machine learning tools can handle** these situations and return a vector of parameters 
⃗
w
 as if there were no collinear features.


In [5]:
from scipy.linalg import lstsq

# Compute OLS using lstsq
w, rss, rank, _ = lstsq(X1, users)

print('w:', w) 
print('rank:', rank) 
print('RSS:', rss) 

w: [155.34445517  27.10638524  31.24446504]
rank: 2
RSS: []


In this code, we call the **lstsq() function with the rank-deficient matrix X1 which returns a set of parameters w,** the rank of the matrix, and the **RSS score which is an array of length zero when the matrix is rank-deficient.** The function also returns the singular values of the matrix (the fourth return value), but we can discard this result by assigning it to the "throwaway" variable _.

Let's compare the performance of this model to a simple linear regression with the 
R
2
 coefficient. We can use the r2_score(y,y_pred) function from the Scikit-learn metrics module.

In [6]:
from sklearn.metrics import r2_score

# R^2 coefficient of simple linear regression
coefs = np.polyfit(temp, users, deg=1)
y_pred_normal = np.polyval(coefs, temp)
r2_normal = r2_score(users, y_pred_normal)
print('R^2 normal:', r2_normal)

# R^2 coefficient with collinear features
y_pred_collinear = np.matmul(X1, w)
r2_collinear = r2_score(users, y_pred_collinear)
print('R^2 collinear:', r2_collinear)

R^2 normal: 0.5954233080185317
R^2 collinear: 0.5954233080185317


**We can see that collinearity didn't affect performance in this case.**

### Nearly collinear features
ometimes, features are highly correlated but there isn't a perfect linear relationship between them. These are nearly collinear features.

For instance, say that we measure temperatures with two different thermometers. One gives temperatures in degrees Celsius and the other in degrees Fahrenheit. To simulate this scenario, we can simply convert the temp_C values to degrees Fahrenheit.

In [14]:
# Convert to degrees Fahrenheit
temp_F = 1.8*temp_C + 32

# Add small variations
noise = np.random.normal(loc=0, scale=0.01, size=temp_F.shape)
temp_F += noise

# Create input matrix X
X = np.c_[temp_C, temp_F]

# Compute OLS using lstsq
X1 = np.c_[np.ones(X.shape[0]), X] # Create X1 matrix
w, rss, rank, _ = lstsq(X1, users) # OLS

print('rank:', rank) # Returns: 3
print('RMSE:', np.sqrt(rss/len(users))) # Depends on the noise value
print('w:', w) # Depends on the noise value

rank: 3
RMSE: 233.28290574613763
w: [19735.20830565  1133.01833753  -611.75287897]


**This time, X1 is full rank and the lstsq() function returns the residual sum of squares. If you run the code several times, you can see that the coefficients vary a lot.**

### This is due to ill-conditioning.
### Ill-conditioning

In the example from above, **X1 has full-rank and the OLS solution exists**. However, it's numerically unstable. A small change in the data produces very different coefficients. We can quantify this phenomenon with the **condition number**. Inverting a matrix with a large condition number is **numerically unstable**.

We can compute the condition number of X1 with the cond() function from the Numpy linalg module.

In [15]:
# Condition number
cn = np.linalg.cond(X1)
print('Condition number:', cn) # Depends on the noise value

Condition number: 207762.50372569985


The value of cn depends on the noise in the X1 matrix from above. However, you should get a value above 200 thousand. **By increasing the scale of the noise, you decrease the correlation between the two variables and you should see that it reduces the condition number.**

Again, ill-conditioning doesn't necessarily affect the predictive accuracy of the model. In most cases, it will simply result in large variations in the model coefficients.

In [16]:
# Same with the nearly collinear matrix
y_pred_nearcol = np.matmul(X1, w)
r2_nearcol = r2_score(users, y_pred_nearcol)

# R^2 coefficient with nearly collinear features
print('R^2 nearly collinear:', r2_nearcol)
# should be around 0.59

R^2 nearly collinear: 0.5957144888659831


### Regularization

One way to solve ill-conditioning is to create a **constraint on the coefficients**. The idea is to modify the **objective function and add a regularization term that penalizes** large coefficients.

Scikit-learn implements regularization with the Ridge estimator which is similar to the LinearRegression one.

In [19]:
from sklearn.linear_model import Ridge

# Add small variations
noise = np.random.normal(loc=0, scale=0.01, size=temp_F.shape)
temp_F = (1.8*temp_C + 32) + noise

# Create input matrix X
X = np.c_[temp_C, temp_F]

# Fit a Ridge regression
ridge = Ridge(alpha=100)
ridge.fit(X, users)

print('Coefficients:', ridge.coef_)
print('Intercept:', ridge.intercept_)
print('R^2:', ridge.score(X, users))

Coefficients: [ 7.46305562 13.51599612]
Intercept: -272.0801819956971
R^2: 0.5954284128001864


In this code, we create an input matrix X of nearly collinear features, and we fit the Ridge model to it. Note that we pass the X matrix to the fit() function (i.e., without the column of ones) since Scikit-learn objects automatically compute the intercept term. Also, we use an alpha parameter which controls the regularization strength. Finally, we print the model coefficients and the 
R
2
 score.

If you run the code several times, you should see that the coefficient are more stable than before.

It's interesting to note that the second run has an 
R
2
 score of 0.595460937325 which is slightly better than the one of the simple linear regression model from above 0.595423308019. This is due to **overfitting - the model fits the noise in the temp_F variable.**

But we will learn more about regularization, overfitting and the Ridge estimator later in this course.

### Summary
Let's summarize what we've learned in this unit. Here are a few takeaways.

- Collinearity happens when there is an **exact linear relationship** between one or more features. In this case, the 
X
 matrix, with the additional column of ones, and its moment matrix 
X
⊺
X
 **are rank-deficient and the OLS solution doesn't exist.**
- Nearly collinear features make **computations numerically unstable** which result in large variations in the model coefficients. This is called **ill-conditioning**. It's numerically unstable to compute the inverse of matrices with a **large condition number**.
- Regularization stabilizes model coefficients.

Ill-conditioning is a complex topic. You can take a look at this thread (https://stats.stackexchange.com/questions/168622/why-is-multicollinearity-not-checked-in-modern-statistics-machine-learning) if you want to learn more about it. However, note that some answers refer to models that we will see later in this course.

In the next exercise, you will fit a linear regression model to a data set with multiple features (including collinear and nearly collinear ones). This is an excellent opportunity to experiment with ill-conditioning.